### Attention Mechanisms
In this notebook, we will sequentially implement different variants of attention mechanisms. These variants will build on each other, with the goal of finally creating a compact, efficient implementation of an attention mechanism, which we can then plug into our LLM architecture.

**Simple Self-Attention**: Introduce the broader idea behind attention.

**Self-Attention**: Trainable weights that forms the basis of the mechanisms used in LLMs.

**Causal Attention**: A self-attention variant allowing a model to consider only previous and current inputs in a sequence, ensuring temporal order during text generation.

**Multi-head Attention**: A self-attention and causal attention extension, which enables the model to simultaneously attend to information from different representation subspaces.

#### Why Attention?
In machine translation, it is not possible to merely translate word by word. The translation process requires contextual understanding and grammatical alignment.

- "Kannst du mir helfen diesen Satz zu uebersetzen" should not be translated to "Can you me help this sentence to translate", but rather to "Can you help me translate this sentence".
- Certain words require access to words appearing before or later in the original sentence. For instance, the verb "to translate" should be used in the context of "this sentence", and not independently.

Typically, to overcome this challenge, deep neural networks with two submodules are used:

- **encoder**: first read in and process the entire text (already done in the `preprocessing.ipynb` notebook).

- **decoder**: produces the translated text.

Pre-LLM architectures typically involved recurrent neural networks, a type of neural network where outputs from previous steps are fed as inputs to the current step, making them well-suited for sequential data. In this many-to-one RNN architecture, the input text is fed token by token into the encoder, which processes it sequentially. The terminal state of the encoder is a memory cell, known as the hidden state, which encodes the entire input. This hidden state is then fed to a decoder that would then generate the translated sentence, word by word, one word at a time.

|     ![RNNEncoder-Decoder](images/RNNencoderdecoder.png)     |
|:-----------------------------------------------------------:|
| *RNN Encoder Decoder* (Dive Into Deep Learning Chapter 10.7 |

- While the encoder is many-to-one, the decoder is a one-to-many architecture, since the hidden state is passed at every step of the decoding process.
- While it is not strictly necessary to understand the inner workings of encoder-decoder RNNs to develop an LLM, the `seq2seq.ipynb` notebook in stage 1 aims to more deeply explore these architectures.

**encoder-decoder RNNs had many shortcomings that motivated the design of attention mechanisms**, namely that the it was not possible to access earlier hidden states from the encoder during the decoding phase, since we rely on a single hidden state containing all the relevant information. Context was lost, especially in complex sentences where dependencies span larger distances.



## Simple Self-Attention

The original *transformer* architecture includes a 'Self-Attention' mechanism inspired by the Badhanau attention mechanism mentioned in `seq2seq.ipynb`.

A mechanism that uses self-attention allows each position in the input sequence (each word), to consider the importance of all other positions (all other words) in the same sequence when creating the embedding of such sequence.

In short, **the goal of a self-attention mechanism is to, for each position in the input sequence, compute a context vector that captures, quantifies, and combines information from all other positions.** For example, given an input vector $X = {x^{(1)}, x^{(2)},..., x^{(T)}}$ (which represents a text that has already been transformed into token embeddings), we want to compute the context vector $z^{(3)}$ of the 3rd position, $x^{(3)}$. The importance of each input for computing $z^{(3)}$ is determined by attention weights ${\alpha_{3,1}, \alpha_{3,2},..., \alpha_{3, T}}$, being $z^{(3)}$ a combination of all input vectors weighted with respect to input element $x^{(3)}$.

- Our goal is to compute context vectors $z^{(i)}$ for all $x^{(i)}$ in the input sequence. The resulting context vectors, we can say, are enriched, more informational embedding vectors.

#### Why 'Self-Attention'

The 'self' refers to the fact that the mechanism computes weights by assessing and learning the dependencies of different positions **within the same sequence**. The relationships of the various parts of the input itself are considered. This is in contrast to attention mechanisms that focus on assessing and learning the relationship of elements that are part of distinct sequences. Sequence-to-sequence models, for instance, where assessment might be done over an input sequence and a distinct output sequence is not a self-attentive mechanism.




In [12]:
import torch
import numpy as np

inputs = torch.tensor(
    [[0.35, 0.15, 0.89], #These (x^1)
     [0.97, 0.8, 0.3], #are (x^2)
     [0.65, 0.34, 0.24], #random (x^3)
     [0.2, 0.87, 0.34], #words (x^4)
     [0.86, 0.13, 0.05], #and (x^5)
     [0.10, 0.20, 0.30]] #embeddings (x^6)
)

print(f'This is the second token: {inputs[1]}')

print(f'We initialize an empty vector to calculate intermediate attention scores, one for each row (word) of our embeddings: {torch.empty(inputs.shape[0])}')

def calculate_intermediate_attention_score_for_one_position(inputs):
    query = inputs[1] # we are working with the second input token as the query
    attn_scores_2 = torch.empty(inputs.shape[0])
    for i, x_i in enumerate(inputs):
        attn_scores_2[i] = torch.dot(query, x_i)
    return attn_scores_2


attention_scores_2 = calculate_intermediate_attention_score_for_one_position(inputs)
print(f'The intermediate attention scores for the second token are: {attention_scores_2}')

This is the second token: tensor([0.9700, 0.8000, 0.3000])
We initialize an empty vector to calculate intermediate attention scores, one for each row (word) of our embeddings: tensor([2.3694e-38, 2.3694e-38, 2.3694e-38, 2.3694e-38, 3.6013e-43, 0.0000e+00])
The intermediate attention scores for the second token are: tensor([0.7265, 1.6709, 0.9745, 0.9920, 0.9532, 0.3470])


#### Dot Product Intuition

In the code above, we have calculated intermediate values for the attention weights of the embedded query token $x^{(2)}$. We have simply initialized an empty, 6-dimension vector (one for each embedded input token) and set each value to be the dot product between input $x^{(i)}$ and query $x^{(2)}$.

Recall that, for two vectors $u = [u_{1}, u_{2}, ..., u_{n}]$ and $v = [v_{1}, v_{2}, ..., v_{3}]$ in $R^{n}$, the dot product is defined as the sum of the element-wise multiplication:

$ u \bullet v = \sum_{i=1}^{n} u_{i}v_{i} = u_{1}v_{1} + u_{2}v_{2} + ... + u_{n}v_{n}$

Alternatively, the dot product can also be geometrically expressed as:

$ u \bullet v = |u||v|cos(\theta)$

where $|u|$ and $|v|$ are the magnitudes of the vectors, and $\theta$ is the angle between them.

- When $\theta = 0^\circ$, the vectors are parallel and point in the same direction. $cos(0) = 1$, and therefore $u \bullet v = |u||v|$. This is the maximum possible value of the dot product for given magnitudes.
- When $\theta = 90^\circ$, the vectors are perpendicular. $cos(90) = 0$, and $u \bullet v = 0$. This indicates no alignment or similarity. The vectors share no directional overlap.
- When $\theta = 180^\circ$, the vectors are parallel but point to opposite directions. $cos(180) = -1$ and $u \bullet v = -|u||v|$. This negative value indicates dissimilarity.

**The dot product depends on both alignment and magnitude.** Without normalization, it is possible that two long vectors at a small angle have a larger dot product than two short, perfectly aligned vectors. Therefore, **vectors are often normalized to provide a range from -1 to 1 as a pure measure of directional similarity**:

$ Cosine Similarity = \frac{u \bullet v}{|u||v|} = cos(\theta)$

- Even without normalization, however, the dot product captures similarity through component-wise agreement (large values in the same positions yield a large sum). **Scaling and normalization might refine this measure, but this core principle holds.**

It is extremely important to understand how dot products determine the extent to which each position in a sequence gives importance to any other element. The higher the dot product, the higher the similarity, and the higher the attention score between the two positions.
